In [ ]:
#| label: libraries
#| include: false
import pandas as pd
import numpy as np
import plotly.express as px

## Elevator pitch

__Analyzing the flights_missing.json dataset offers insights into flight delays and associated factors such as airline performance and weather conditions. By exploring the data, we can uncover patterns in the frequency and duration of flight delays, identify which airlines are most prone to delays, and assess the impact of weather on flight schedules. Utilizing visualizations, we can depict correlations between delay durations and various parameters like time of day, day of the week, and weather conditions, enabling stakeholders to make informed decisions for improving flight operations and passenger experiences.__


In [ ]:
#| label: project-data
#| code-summary: Read and format project data

# Include and execute your code here
df = pd.read_json("https://github.com/byuidatascience/data4missing/raw/master/data-raw/flights_missing/flights_missing.json")

## QUESTION|TASK 1

__Fix all of the varied missing data types in the data to be consistent (all missing values should be displayed as “NaN”). In your report include one record example (one row) from your new data, in the raw JSON format. Your example should display the “NaN” for at least one missing value.__

_type your results and analysis here_


In [ ]:
#| label: Q1
#| code-summary: Read and format data
# Include and execute your code here

df.replace({
    -999: np.nan,
    'n/a': np.nan,
    'Febuary': 'February',
    '': np.nan,
    '1500+': 1500
}, inplace=True)

df = df.query("month != 'na'")

avg_delay = df['num_of_delays_late_aircraft'].mean()

df['num_of_delays_late_aircraft'].replace(np.nan, avg_delay, inplace=True)

df.iloc[2]  # third row (index 2)

## QUESTION|TASK 2

__Which airport has the worst delays? Discuss the metric you chose, and why you chose it to determine the “worst” airport. Your answer should include a summary table that lists (for each airport) the total number of flights, total number of delayed flights, proportion of delayed flights, and average delay time in hours.__

_type your results and analysis here_


In [ ]:
#| label: Q2
#| code-summary: Read and format data
# Include and execute your code here

q2 = (df.groupby('airport_code').agg(
total_flights=('num_of_flights_total', 'sum'),
delayed_flights=('num_of_delays_total', 'sum'),
avg_delay_time=('minutes_delayed_total', 'mean')))

q2['avg_delay_hours'] = q2['avg_delay_time'] / 60


q2['is_delayed'] = q2['delayed_flights'] / q2['total_flights'] *100

q2 = q2.sort_values(by='is_delayed', ascending=False)

# table
print(q2)

## QUESTION|TASK 3

__What is the best month to fly if you want to avoid delays of any length? Discuss the metric you chose and why you chose it to calculate your answer. Include one chart to help support your answer, with the x-axis ordered by month. (To answer this question, you will need to remove any rows that are missing the Month variable.)__

_type your results and analysis here_


In [ ]:
#| label: Q3
#| code-summary: Read and format data
# Include and execute your code here

df_months = df.dropna(subset=['month'])

q3 = df.groupby('month').agg(
    total_flights=('num_of_flights_total', 'sum'),
    delayed_flights=('num_of_delays_total', 'sum')
)

#proportion
q3['is_delayed'] = q3['delayed_flights'] / q3['total_flights']*100


months_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
q3 = q3.reindex(months_order)

q3_visual = px.bar(q3, x=q3.index, y='is_delayed', 
                   color='is_delayed',
                   labels={'is_delayed': 'Delayed Percentage'},
                   title='Proportion of Delayed Flights by Month')
q3_visual.update_layout(xaxis_title='Month', yaxis_title='Proportion of Delayed Flights')

q3_visual.show()

## QUESTION|TASK 4

__According to the BTS website, the “Weather” category only accounts for severe weather delays. Mild weather delays are not counted in the “Weather” category, but are actually included in both the “NAS” and “Late-Arriving Aircraft” categories. Your job is to create a new column that calculates the total number of flights delayed by weather (both severe and mild). You will need to replace all the missing values in the Late Aircraft variable with the mean. Show your work by printing the first 5 rows of data in a table. Use these three rules for your calculations:__

_type your results and analysis here_


In [ ]:
#| label: Q3
#| code-summary: Read and format data
# Include and execute your code here


__100% of delayed flights in the Weather category are due to weather__

In [ ]:
#| label: Q3
#| code-summary: Read and format data
# Include and execute your code here


__30% of all delayed flights in the Late-Arriving category are due to weather.__

In [ ]:
#| label: Q3
#| code-summary: Read and format data
# Include and execute your code here


__From April to August, 40% of delayed flights in the NAS category are due to weather. The rest of the months, the proportion rises to 65%.__

In [ ]:
#| label: Q3
#| code-summary: Read and format data
# Include and execute your code here

df['Late Aircraft'].fillna(df['Late Aircraft'].mean(), inplace=True)

df['total_weather_delays'] = df['Weather']
df['total_weather_delays'] += df['Late-Arriving'] * 0.3

df['total_weather_delays'] += np.where((df['Month'] >= 4) & (df['Month'] <= 8), df['NAS'] * 0.4, df['NAS'] * 0.65)

print(df.head())

## QUESTION|TASK 5

__Using the new weather variable calculated above, create a barplot showing the proportion of all flights that are delayed by weather at each airport. Discuss what you learn from this graph.__

_type your results and analysis here_


In [ ]:
#| label: Q3
#| code-summary: Read and format data
# Include and execute your code here

q4['total_weather_delays'] = q4['nas_weather_delays'] + q4['mild_weather_delays']

total_flights = df.groupby('airport_code')['num_of_flights_total'].sum().reset_index()

q5 = q4.merge(total_flights, on='airport_code', how='left')
q5['is_delayed_by_weather'] = (q5['total_weather_delays'] / total_flights['num_of_flights_total'])*100


q5_visual = px.bar(q5, x='airport_code', y='is_delayed_by_weather',
title='Proportion of Flights Delayed by Weather at Each Airport',
labels={'weather_delay_proportion': 'Proportion of Flights Delayed by Weather', 'airport': 'Airport'})
q5_visual.update_xaxes(title_text='Airport')
q5_visual.update_yaxes(title_text='Proportion of Flights Delayed by Weather')

q5_visual.show()